# Representação de dados

Para que uma informação seja representada em bytes, é preciso que uma convenção de representação seja seguida.

## Conversões

Nos exemplos seguintes, são apresentadas algumas alternativas para a codificação:

* Cadeias de caracteres: uso de conjunto de caracteres UTF-8
* Valores inteiros: uso de valor binário com sinal de 4 bytes de comprimento, *big endian*
* Valores reais: uso do padrão IEEE 754 de dupla precisão (8 bytes)

In [5]:
# Cadeias de caracteres
cadeia = "Computação"
bytes_cadeia = bytes(cadeia, "utf-8")
print(f"{cadeia} --> {bytes_cadeia}")

# Inteiros
inteiro = 2500
bytes_inteiro = inteiro.to_bytes(4, "big", signed = True)
print(f"{inteiro} --> {bytes_inteiro}")

# Reais
from struct import pack
real = -3.1415666
bytes_real = pack("d", real)
print(f"{real} --> {bytes_real}")

Computação --> b'Computa\xc3\xa7\xc3\xa3o'
2500 --> b'\x00\x00\t\xc4'
-3.1415666 --> b'\xb6\xa5i\xab\xed!\t\xc0'


As conversões inversas permitem recuperar os valores originais.

In [7]:
# Cadeia
print(f"{bytes_cadeia} --> {bytes_cadeia.decode('utf-8')}")

# Inteiro
print(f"{bytes_inteiro} --> {int.from_bytes(bytes_inteiro, 'big', signed = True)}")

# Real
from struct import unpack
print(f"{bytes_real} --> {unpack('d', bytes_real)[0]}")

b'Computa\xc3\xa7\xc3\xa3o' --> Computação
b'\x00\x00\t\xc4' --> 2500
b'\xb6\xa5i\xab\xed!\t\xc0' --> -3.1415666


## Dados em arquivos

Agora é hora de gravar os dados em um arquivo (binário, é claro)!

Para tanto, são gravadas as representações dos dados em bytes.

In [8]:
# Criação de um arquivo binário
try:
  arquivo = open("dados.dat", "wb")
except IOError as excecao:
  print(f"Erro de acesso: {excecao}")
else:
  # Gravação dos dados binários
  comprimento = arquivo.write(bytes_cadeia)
  print(f"{comprimento} bytes escritos")
  comprimento = arquivo.write(bytes_inteiro)
  print(f"{comprimento} bytes escritos")
  comprimento = arquivo.write(bytes_real)
  print(f"{comprimento} bytes escritos")

  # Encerramento do acesso ao arquivo
  arquivo.close()
  print()

# Visualização do conteúdo do arquivo
!ls -l dados.dat
!hd dados.dat

12 bytes escritos
4 bytes escritos
8 bytes escritos

-rw-r--r-- 1 root root 24 Nov 18 18:13 dados.dat
00000000  43 6f 6d 70 75 74 61 c3  a7 c3 a3 6f 00 00 09 c4  |Computa....o....|
00000010  b6 a5 69 ab ed 21 09 c0                           |..i..!..|
00000018


Agora, a recuperação dos dados, incluindo as conversões para a representação original.

In [11]:
# Acesso ao arquivo binário
try:
  arquivo = open("dados.dat", "rb")
except IOError as excecao:
  print(f"Erro de acesso: {excecao}")
else:
  bytes_dados = arquivo.read(4)  # 12 bytes da string
  print(bytes_dados.decode("utf-8"))

  bytes_dados = arquivo.read(4)  # 4 bytes do inteiro
  print(int.from_bytes(bytes_dados, "big", signed = True))

  bytes_dados = arquivo.read(8)  # 8 bytes do real
  print(unpack("d", bytes_dados)[0])

Comp
1970561475
-5.7646090573986456e+19


## Exemplo: arquivo de nomes

Segue um exemplo de criação de um arquivo de nomes.

In [12]:
# Criação de um arquivo binário
try:
  arquivo = open("nomes.dat", "wb")
except IOError as excecao:
  print(f"Erro de acesso: {excecao}")
else:
  # Entrada de dados até linha vazia
  nome = input("Nome: ")
  while nome != "":
    # Conversão e gravação
    bytes_nome = bytes(nome, "utf-8")
    comprimento = arquivo.write(bytes_nome)
    print(f"{comprimento} bytes escritos")

    # Próxima leitura
    nome = input("Nome: ")

  # Encerramento do acesso ao arquivo
  arquivo.close()
  print()

Nome: Peter
5 bytes escritos
Nome: Parker
6 bytes escritos
Nome: Spider
6 bytes escritos
Nome: Man
3 bytes escritos
Nome: André
6 bytes escritos
Nome: 



In [13]:
# Visualização do conteúdo do arquivo
!ls -l nomes.dat
!hd nomes.dat
!cat nomes.dat

-rw-r--r-- 1 root root 26 Nov 18 18:19 nomes.dat
00000000  50 65 74 65 72 50 61 72  6b 65 72 53 70 69 64 65  |PeterParkerSpide|
00000010  72 4d 61 6e 41 6e 64 72  c3 a9                    |rManAndr..|
0000001a
PeterParkerSpiderManAndré

# Campos

Campo é a menor unidade de informação.

Para programar com campos, foram implementadas classes para facilitar as conversões, gravações e leituras em arquivos.

A implementação do pacote [estrutarq](https://github.com/jandermoreira/estrutarq) está disponível no GitHub.

In [ ]:
!# Execute para baixar e atualizar o código disponível!
!git clone http://github.com/jandermoreira/estrutarq || \
  echo "Já foi executado antes? Basta baixar só uma vez..."
!cd estrutarq; git pull

## Campos com terminadores

Os limites de um campo podem ser determinados pelo uso de um terminador que indique o fim dos bytes de dados. A representação dos dados é sempre textual, inclusive para campos núméricos.

In [17]:
# Exemplos de campos em formato textual com terminadores
from estrutarq.campo import CampoCadeiaTerminador, CampoRealTerminador

# Campo cadeia de caracteres
campo_nome = CampoCadeiaTerminador()  # terminador é 0x00
campo_nome.valor = "José João Victório Magalhães"
print(f"Nome: {campo_nome.valor}")
print(f" * {campo_nome.dado_formatado}")

# Campo real
campo_salario = CampoRealTerminador()
campo_salario.valor = 5889.20
print(f"Salário: {campo_salario.valor:.2f}")
print(f" * {campo_salario.dado_formatado}")

Nome: José João Victório Magalhães
 * b'Jos\xc3\xa9 Jo\xc3\xa3o Vict\xc3\xb3rio Magalh\xc3\xa3es\x00'
Salário: 5889.20
 * b'5889.2\x00'


## Campos prefixados pelo comprimento

O prefixo adotado para o comprimento é de dois bytes (16 bits), usando um número inteiro binário sem sinal. Desta forma, o comprimento máximo é $2^{16}$ bytes.

In [21]:
# Exemplos de campos em formato textual com prefixo de comprimento em binário
from estrutarq.campo import CampoIntPrefixado, CampoRealPrefixado, CampoCadeiaPrefixado

# Campo inteiro
campo_idade = CampoIntPrefixado()
campo_idade.valor = -987654779
print(f"Idade: {campo_idade.valor}")
print(f" * {campo_idade.dado_formatado}")

# Campo cadeia
campo_cadeia = CampoCadeiaPrefixado()
campo_cadeia.valor =  "A" * 2048  # cadeia longa (2KiB)
print(f"Idade: {campo_cadeia.valor}")
print(f" * {campo_cadeia.dado_formatado}")

# Campo real
campo_salario = CampoRealPrefixado()
campo_salario.valor = 5889.20
print(f"Salário: {campo_salario.valor:.2f}")
print(f" * {campo_salario.dado_formatado}")

Idade: -987654779
 * b'\x00\n-987654779'
Idade: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

## Campos de comprimento fixo

Um campo de comprimento fixo é definido por um comprimento prederminado de bytes, o que pode incorrer em truncamentos.

O campo fica com uma área útil (com os dados) e com possível fragmentação (disperdício). A parte não usada usa um byte de preenchimento para indicar que não é dado válido. O preenchimento dos campos é feito com o byte 0xFF.

In [22]:
# Exemplos de campos em formato textual com comprimento fixo
from estrutarq.campo import CampoDataFixo, CampoIntFixo, CampoRealFixo

# Campo inteiro
campo_idade = CampoIntFixo(15)  # 15 bytes
campo_idade.valor = -987654
print(f"Idade: {campo_idade.valor}")
print(f" * {campo_idade.dado_formatado}")

# Campo data
campo_data = CampoDataFixo()  # datas usam sempre 10 bytes de comprimento
campo_data.valor = "2022-06-14"
print(f"Data: {campo_data.valor}")
print(f" * {campo_data.dado_formatado}")

# Campo real
campo_salario = CampoRealFixo(12)  # 12 bytes
campo_salario.valor = 5889.20
print(f"Salário: {campo_salario.valor:.2f}")
print(f" * {campo_salario.dado_formatado}")

Idade: -987654
 * b'-987654\xff\xff\xff\xff\xff\xff\xff\xff'
Data: 2022-06-14
 * b'2022-06-14'
Salário: 5889.20
 * b'5889.2\xff\xff\xff\xff\xff\xff'


## Campos binários

Para alguns campos, a representação binária é mais natural: é o caso de inteiros e reais. Tempo (hora e data) também podem usar a representação binária.

In [23]:
# Exemplos de campos em formato binário
from estrutarq.campo import CampoHoraBinario, CampoIntBinario, CampoRealBinario


# Campo inteiro
campo_idade = CampoIntBinario()
campo_idade.valor = -987654
print(f"Idade: {campo_idade.valor}")
print(f" * {campo_idade.dado_formatado}")

# Campo data
campo_hora = CampoHoraBinario()
campo_hora.valor = "13:45:00"
print(f"Hora: {campo_hora.valor}")
print(f" * {campo_hora.dado_formatado}")

# Campo real
campo_salario = CampoRealBinario()
campo_salario.valor = 5889.20
print(f"Salário: {campo_salario.valor:.2f}")
print(f" * {campo_salario.dado_formatado}")

Idade: -987654
 * b'\xff\xff\xff\xff\xff\xf0\xed\xfa'
Hora: 13:45:00
 * b'\xff\xff\xff\xff|VB\xdc'
Salário: 5889.20
 * b'33333\x01\xb7@'


# Arquivos com campos

Uma vez escolhida a forma de representação dos dados, um arquivo com os campos pode ser criado, obedecidas as formas de organização de campos.

Independentemente da forma de organização ou se os dados são textuais ou binários, os aquivos são sempre acessados como binários, lendo ou escrevendo sequências de bytes.

Segue, assim, um exemplo de criação de um arquivo para guardar uma matriz de valores reais.

O arquivo é organizado da seguinte forma:
* Dois valores inteiros textuais com terminador, sendo o primeiro o número de linhas e o segundo, o número de colunas
* Uma descrição textual da matriz (prefixada pelo comprimento)
* Os valores reais de cada elemento da matriz, em formato binário


In [24]:
# Criação de uma matriz
import numpy

numero_linhas = 10
numero_colunas = 6
matriz = numpy.random.rand(numero_linhas, numero_colunas) * 100
print(matriz)

[[14.71156097 47.17430677 48.71977689 59.28197545 53.49506559 34.18811678]
 [ 8.42987671 49.05112333 84.36514718 17.55716761 64.38473755 94.63151785]
 [44.64712281 25.06884897  1.4592491  46.69560103 31.52951247 30.55989364]
 [98.87997548 34.42216584 58.01827165 99.80513135 70.6069825  55.21517055]
 [18.65192536 93.33249942 90.30213289 40.8002952   4.48364473 85.0583021 ]
 [73.72204008 13.05054031 99.41590312 53.78043309 20.11522393 39.71280872]
 [10.77929571 92.74253062 70.62426713 19.71719656 34.26499883 10.51119029]
 [55.23610302 99.46888633 73.71652582 39.33195985 63.63375349  5.90800533]
 [95.44368678 98.54746444 61.90964437  8.68051222 22.25962798 20.34028127]
 [19.3885365  45.46062406 49.70961233 94.54884007 92.68884129 12.63202484]]


In [29]:
# Escrita da matriz em um arquivo
from estrutarq.campo import CampoIntTerminador, CampoCadeiaPrefixado
try:
  arquivo = open("matriz", "wb")
except IOError as excecao:
  print(f"Erro: {excecao}")
else:
  # Dimensões
  dimensao = CampoIntTerminador()
  dimensao.valor = numero_linhas
  dimensao.escreva(arquivo)
  dimensao.valor = numero_colunas
  dimensao.escreva(arquivo)
  print("Dimensões")

  # Comentário
  comentario = CampoCadeiaPrefixado()
  comentario.valor = "Criado por mim, hoje"
  comentario.escreva(arquivo)
  print("Comentário")

  # Elementos
  elemento = CampoRealBinario()
  for i in range(numero_linhas):
    for j in range(numero_colunas):
      elemento.valor = matriz[i][j]
      elemento.escreva(arquivo)
  print("Elementos")

  arquivo.close()

Dimensões
Comentário
Elementos


In [28]:
!hd matriz

00000000  00 00 00 00 00 00 00 0a  00 00 00 00 00 00 00 06  |................|
00000010  00 14 43 72 69 61 64 6f  20 70 6f 72 20 6d 69 6d  |..Criado por mim|
00000020  2c 20 68 6f 6a 65 3a ab  14 b8 51 6c 2d 40 92 3d  |, hoje:...Ql-@.=|
00000030  26 af 4f 96 47 40 72 72  28 a6 21 5c 48 40 dc 84  |&.O.G@rr(.!\H@..|
00000040  86 c5 17 a4 4d 40 46 dd  31 4f 5e bf 4a 40 b0 4e  |....M@F.1O^.J@.N|
00000050  ef 35 14 18 41 40 4f 3f  f4 cc 18 dc 20 40 eb ba  |.5..A@O?.... @..|
00000060  88 35 8b 86 48 40 7f 33  42 92 5e 17 55 40 38 19  |.5..H@.3B.^.U@8.|
00000070  62 89 a2 8e 31 40 a5 cd  42 8a 9f 18 50 40 05 00  |b...1@..B...P@..|
00000080  dc c9 6a a8 57 40 1d 9a  8c eb d4 52 46 40 a1 d7  |..j.W@.....RF@..|
00000090  fd 15 a0 11 39 40 f6 fc  c4 94 15 59 f7 3f ae 7d  |....9@.....Y.?.}|
000000a0  5b 74 09 59 47 40 4b 6c  26 21 8e 87 3f 40 33 12  |[t.YG@Kl&!..?@3.|
000000b0  8a 30 55 8f 3e 40 51 a4  ab 84 51 b8 58 40 25 7c  |.0U.>@Q...Q.X@%||
000000c0  c3 87 09 36 41 40 e3 c8  ad b9 56 02 4d 40

Finalmente, a recuperação da matriz a partir do arquivo

In [30]:
# Leitura da matriz em um arquivo
try:
  arquivo = open("matriz", "rb")
except IOError as excecao:
  print(f"Erro: {excecao}")
else:
  # Dimensões
  dimensao = CampoIntTerminador()
  dimensao.leia(arquivo)
  numero_linhas = dimensao.valor
  dimensao.leia(arquivo)
  numero_colunas = dimensao.valor
  print(f"Matriz: {numero_linhas} x {numero_colunas}")

  # Comentário
  comentario = CampoCadeiaPrefixado()
  comentario.leia(arquivo)
  print(f"Comentário: {comentario.valor}")

  # Criação da matriz e leitura dos elementos
  nova_matriz = numpy.zeros((numero_linhas, numero_colunas))
  elemento = CampoRealBinario()
  for i in range(numero_linhas):
    for j in range(numero_colunas):
      elemento.leia(arquivo)
      nova_matriz[i][j] = elemento.valor
    print()

  arquivo.close()

  print(nova_matriz)

Matriz: 10 x 6
Comentário: Criado por mim, hoje










[[14.71156097 47.17430677 48.71977689 59.28197545 53.49506559 34.18811678]
 [ 8.42987671 49.05112333 84.36514718 17.55716761 64.38473755 94.63151785]
 [44.64712281 25.06884897  1.4592491  46.69560103 31.52951247 30.55989364]
 [98.87997548 34.42216584 58.01827165 99.80513135 70.6069825  55.21517055]
 [18.65192536 93.33249942 90.30213289 40.8002952   4.48364473 85.0583021 ]
 [73.72204008 13.05054031 99.41590312 53.78043309 20.11522393 39.71280872]
 [10.77929571 92.74253062 70.62426713 19.71719656 34.26499883 10.51119029]
 [55.23610302 99.46888633 73.71652582 39.33195985 63.63375349  5.90800533]
 [95.44368678 98.54746444 61.90964437  8.68051222 22.25962798 20.34028127]
 [19.3885365  45.46062406 49.70961233 94.54884007 92.68884129 12.63202484]]
